In [7]:
import pyspark
import os
import sys
from pyspark import SparkContext
from pyspark import SparkConf
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
from pyspark.sql import SparkSession

spark = SparkSession.builder.config("spark.driver.memory", "16g").appName('square').getOrCreate()



/home/lplab/anaconda3/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


In [8]:
import pandas as pd
from pyspark.sql import functions as F
df_pd = pd.DataFrame(
data={'integers': [1, 2, 3],
'floats': [-1.0, 0.5, 2.7],
'integer_arrays': [[1, 2], [3, 4, 5], [6, 7, 8, 9]]}
)
df = spark.createDataFrame(df_pd)
df.printSchema() # It will print the Schema
df.show()

root
 |-- integers: long (nullable = true)
 |-- floats: double (nullable = true)
 |-- integer_arrays: array (nullable = true)
 |    |-- element: long (containsNull = true)

+--------+------+--------------+
|integers|floats|integer_arrays|
+--------+------+--------------+
|       1|  -1.0|        [1, 2]|
|       2|   0.5|     [3, 4, 5]|
|       3|   2.7|  [6, 7, 8, 9]|
+--------+------+--------------+



In [9]:
from pyspark.sql.functions import udf
@udf
def square(x):
    return x*x

In [1]:
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import udf

# Define the square function
def square(x):
    return x * x

# Create a Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# Sample DataFrame
data = [(1, 1.5), (2, 2.5), (3, 3.5)]
columns = ["integers", "floats"]
df = spark.createDataFrame(data, columns)

# Define the UDF using the square function
square_udf_int = F.udf(lambda z: square(z), IntegerType())

# Use the UDF in DataFrame transformation
result_df = df.select('integers',
                      'floats',
                      square_udf_int('integers').alias('int_squared'),
                      square_udf_int('floats').alias('float_squared'))

# Show the result
result_df.show()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/05 14:42:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/01/05 14:42:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


+--------+------+-----------+-------------+
|integers|floats|int_squared|float_squared|
+--------+------+-----------+-------------+
|       1|   1.5|          1|         NULL|
|       2|   2.5|          4|         NULL|
|       3|   3.5|          9|         NULL|
+--------+------+-----------+-------------+

